<a href="https://colab.research.google.com/github/boobopbiboop/burn-wound-measurement/blob/main/step_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [23]:
import cv2
import numpy as np
import json
from pathlib import Path
from google.colab import drive
import os
import pandas as pd
from tqdm.notebook import tqdm
from glob import glob
import time

# STEP 1: MOUNT DRIVE & FIND SHARED FOLDER

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
BASE_PATH       = '/content/drive/MyDrive/BurnDetection_ProcessedDataset'
AUGMENTED_DIR   = os.path.join(BASE_PATH, 'augmented')
SEGMENTED_DIR   = os.path.join(BASE_PATH, 'segmented')
MEASUREMENT_DIR = os.path.join(BASE_PATH, 'measured_copy')

In [24]:
# Buat subfolder untuk setiap method
METHODS = ['gray', 'binary_gray', 'binary_hsv', 'opening', 'closing', 'hole_filled', 'contour']

for method in METHODS:
    method_dir = os.path.join(MEASUREMENT_DIR, method)
    os.makedirs(method_dir, exist_ok=True)

In [32]:
# NEW: Folder untuk skipped files
SKIPPED_DIR = os.path.join(MEASUREMENT_DIR, 'skipped')
os.makedirs(SKIPPED_DIR, exist_ok=True)

In [12]:
def get_files(folder):
    return sorted([f for f in glob(os.path.join(folder, "*.*"))
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

augmented_files = get_files(AUGMENTED_DIR)
segmented_files = get_files(SEGMENTED_DIR)

In [13]:
print(f"Augmented  → {len(augmented_files)} gambar")
print(f"Segmented  → {len(segmented_files)} mask")
print(f"Measured   → {len(os.listdir(MEASUREMENT_DIR)) if os.path.exists(MEASUREMENT_DIR) else 0} file tersimpan")

Augmented  → 8464 gambar
Segmented  → 59248 mask
Measured   → 0 file tersimpan


# MEASUREMENT FUNCTIONS

Deteksi koin referensi untuk kalibrasi skala

In [33]:
def detect_coin_for_calibration(image, coin_diameter_cm=2):
    """Deteksi koin referensi dengan filter posisi"""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, w = image.shape[:2]

    lower_gold = np.array([10, 100, 100])
    upper_gold = np.array([30, 255, 255])
    mask = cv2.inRange(hsv, lower_gold, upper_gold)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    best_circle = None
    max_circularity = 0

    margin = 0.2
    corner_regions = [
        (0, 0, w*margin, h*margin),
        (w*(1-margin), 0, w, h*margin),
        (0, h*(1-margin), w*margin, h),
        (w*(1-margin), h*(1-margin), w, h)
    ]

    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 100:
            continue

        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue

        circularity = 4 * np.pi * area / (perimeter * perimeter)

        if circularity > 0.7:
            (x, y), radius = cv2.minEnclosingCircle(contour)

            is_in_corner = False
            for x1, y1, x2, y2 in corner_regions:
                if x1 <= x <= x2 and y1 <= y <= y2:
                    is_in_corner = True
                    break

            score = circularity
            if is_in_corner:
                score *= 2

            if score > max_circularity:
                max_circularity = score
                best_circle = {
                    'center': (int(x), int(y)),
                    'radius': int(radius),
                    'diameter_px': int(radius * 2),
                    'circularity': circularity,
                    'in_corner': is_in_corner
                }

    if best_circle:
        pixels_per_cm = best_circle['diameter_px'] / coin_diameter_cm
        best_circle['pixels_per_cm'] = pixels_per_cm
        best_circle['diameter_cm'] = coin_diameter_cm
        return pixels_per_cm, best_circle

    return None, None

In [34]:
def check_image_quality(image, mask, coin_info):
    """
    NEW: Quality check untuk skip gambar buruk

    Returns:
        is_valid (bool): True jika gambar valid
        skip_reason (str): Alasan skip jika invalid
        quality_score (float): Score 0-100
    """
    reasons = []
    scores = []

    # CHECK 1: Coin detection
    if coin_info is None:
        reasons.append("No coin detected")
        scores.append(0)
    else:
        if coin_info.get('in_corner', False):
            scores.append(100)  # Perfect
        else:
            reasons.append("Coin in center (not corner)")
            scores.append(50)  # Warning but not skip

    # CHECK 2: Mask quality - ada wound area atau tidak
    mask_area = np.sum(mask > 0)
    mask_percentage = (mask_area / mask.size) * 100

    if mask_area < 100:  # Terlalu kecil
        reasons.append(f"Mask too small ({mask_area} pixels)")
        scores.append(0)
    elif mask_percentage > 80:  # Terlalu besar (kemungkinan error)
        reasons.append(f"Mask too large ({mask_percentage:.1f}% of image)")
        scores.append(0)
    else:
        scores.append(100)

    # CHECK 3: Image brightness (blur detection sederhana)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)

    if brightness < 30:  # Terlalu gelap
        reasons.append(f"Image too dark (brightness: {brightness:.1f})")
        scores.append(30)
    elif brightness > 225:  # Terlalu terang/overexposed
        reasons.append(f"Image too bright (brightness: {brightness:.1f})")
        scores.append(30)
    else:
        scores.append(100)

    # CHECK 4: Blur detection (Laplacian variance)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()

    if laplacian_var < 50:  # Terlalu blur
        reasons.append(f"Image too blurry (variance: {laplacian_var:.1f})")
        scores.append(30)
    else:
        scores.append(100)

    # Calculate final score
    quality_score = np.mean(scores)

    # Decision: Skip jika score < 60 ATAU ada critical issue
    critical_issues = ["No coin detected", "Mask too small", "Mask too large"]
    has_critical = any(issue in reasons for issue in critical_issues)

    is_valid = quality_score >= 60 and not has_critical
    skip_reason = "; ".join(reasons) if reasons else "OK"

    return is_valid, skip_reason, quality_score


Hitung luas area luka dalam cm²

In [35]:
def calculate_wound_area(binary_mask, pixels_per_cm):
    """Hitung luas area luka dalam cm²"""
    area_pixels = np.sum(binary_mask > 0)

    if pixels_per_cm is None or pixels_per_cm == 0:
        pixels_per_cm = 50

    area_cm2 = area_pixels / (pixels_per_cm ** 2)

    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        perimeter_px = cv2.arcLength(largest_contour, True)
        perimeter_cm = perimeter_px / pixels_per_cm

        x, y, w, h = cv2.boundingRect(largest_contour)
        width_cm = w / pixels_per_cm
        height_cm = h / pixels_per_cm
    else:
        perimeter_cm = 0
        width_cm = 0
        height_cm = 0

    area_info = {
        'area_pixels': int(area_pixels),
        'area_cm2': round(area_cm2, 2),
        'perimeter_cm': round(perimeter_cm, 2),
        'width_cm': round(width_cm, 2),
        'height_cm': round(height_cm, 2),
        'pixels_per_cm': round(pixels_per_cm, 2)
    }

    return area_info

Hitung rasio area luka terhadap referensi

In [36]:
def calculate_wound_ratio(wound_area_cm2, reference_area_cm2=None):
    """Hitung rasio area luka terhadap referensi"""
    if reference_area_cm2 is None:
        reference_area_cm2 = np.pi * (1 ** 2)

    ratio = wound_area_cm2 / reference_area_cm2

    return {
        'wound_area_cm2': wound_area_cm2,
        'reference_area_cm2': round(reference_area_cm2, 2),
        'ratio': round(ratio, 2),
        'percentage': round(ratio * 100, 2)
    }


In [37]:
def create_visualization(original, mask, coin_info, area_info, ratio_info, method_name):
    """Create visualization with method label"""
    result_img = original.copy()

    if coin_info:
        center = coin_info['center']
        radius = coin_info['radius']
        coin_color = (255, 0, 0) if coin_info.get('in_corner', False) else (0, 165, 255)

        cv2.circle(result_img, center, radius, coin_color, 2)
        cv2.putText(result_img, f"{coin_info['diameter_cm']}cm REF",
                   (center[0]-30, center[1]-radius-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, coin_color, 2)

    mask_colored = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(result_img, 0.7, mask_colored, 0.3, 0)

    y_offset = 30
    texts = [
        f"Method: {method_name.upper()}",
        f"Area: {area_info['area_cm2']} sq.cm",
        f"Perimeter: {area_info['perimeter_cm']} cm",
        f"Size: {area_info['width_cm']}x{area_info['height_cm']} cm",
        f"Ratio: {ratio_info['ratio']}x ({ratio_info['percentage']}%)"
    ]

    if coin_info is None:
        texts.insert(1, "WARNING: No coin detected")
        text_color = (0, 0, 255)
    elif not coin_info.get('in_corner', False):
        texts.insert(1, "WARNING: Coin in center")
        text_color = (0, 165, 255)
    else:
        text_color = (0, 255, 0)

    for i, text in enumerate(texts):
        y_pos = y_offset + (i * 30)
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]

        cv2.rectangle(overlay, (5, y_pos - 22), (15 + text_size[0], y_pos + 5), (0, 0, 0), -1)
        cv2.putText(overlay, text, (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 2)

    return overlay


Pipeline lengkap: deteksi koin, segmentasi, dan pengukuran area

In [38]:
def process_image_with_measurement(image_path, segmented_mask_path, output_dir, method_name, enable_quality_check=True):
    """
    Pipeline measurement dengan quality check

    Returns:
        result: measurement result jika valid
        None: jika di-skip karena quality
    """
    original = cv2.imread(image_path)
    mask = cv2.imread(segmented_mask_path, cv2.IMREAD_GRAYSCALE)

    if original is None or mask is None:
        return None

    # Auto-resize mask jika ukuran tidak sama
    if original.shape[:2] != mask.shape[:2]:
        mask = cv2.resize(mask, (original.shape[1], original.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Detect coin dari ORIGINAL
    pixels_per_cm, coin_info = detect_coin_for_calibration(original)

    # NEW: Quality check
    if enable_quality_check:
        is_valid, skip_reason, quality_score = check_image_quality(original, mask, coin_info)

        if not is_valid:
            # Return skip info
            return {
                'filename': Path(image_path).name,
                'method': method_name,
                'skipped': True,
                'skip_reason': skip_reason,
                'quality_score': round(quality_score, 2)
            }

    # Calculate area dari MASK
    area_info = calculate_wound_area(mask, pixels_per_cm)

    # Calculate ratio
    if coin_info:
        coin_area_cm2 = np.pi * (coin_info['diameter_cm'] / 2) ** 2
        ratio_info = calculate_wound_ratio(area_info['area_cm2'], coin_area_cm2)
    else:
        ratio_info = calculate_wound_ratio(area_info['area_cm2'])

    measurement_result = {
        'filename': Path(image_path).name,
        'method': method_name,
        'skipped': False,
        'coin_detected': coin_info is not None,
        'coin_info': coin_info,
        'area_info': area_info,
        'ratio_info': ratio_info
    }

    # Visualize
    result_img = create_visualization(original, mask, coin_info, area_info, ratio_info, method_name)

    # Save result
    output_path = Path(output_dir) / f"measured_{Path(image_path).name}"
    cv2.imwrite(str(output_path), result_img)

    measurement_result['output_path'] = str(output_path)

    return measurement_result

Batch processing untuk measurement semua gambar

In [39]:
def batch_area_measurement_multi_method(original_dir, segmented_dir, output_base_dir, methods, enable_quality_check=True):
    """
    Batch processing SEMUA METHOD dengan quality filtering
    """
    all_results = {}
    all_stats = {}
    all_skipped = []

    print(f"{'='*70}")
    print(f"MULTI-METHOD MEASUREMENT WITH QUALITY FILTERING")
    print(f"{'='*70}")
    print(f"Methods to process: {len(methods)}")
    print(f"Quality check: {'ENABLED ✅' if enable_quality_check else 'DISABLED ❌'}")
    print(f"{'='*70}\n")

    for method in methods:
        print(f"\n{'='*70}")
        print(f"Processing Method: {method.upper()}")
        print(f"{'='*70}")

        # Setup paths
        method_output_dir = os.path.join(output_base_dir, method)
        os.makedirs(method_output_dir, exist_ok=True)

        # Find segmented files
        prefix = f"{method}_"
        segmented_files = list(Path(segmented_dir).glob(f"{prefix}*.jpg"))

        if not segmented_files:
            print(f"⚠️  No files found with prefix '{prefix}' - SKIPPING")
            continue

        results = []
        stats = {
            'method': method,
            'total': len(segmented_files),
            'success': 0,
            'skipped': 0,
            'failed': 0,
            'coin_detected': 0,
            'coin_in_corner': 0,
            'coin_in_center': 0,
            'no_coin': 0,
            'skip_reasons': {}
        }

        print(f"Files found: {len(segmented_files)}")

        pbar = tqdm(segmented_files, desc=f"📏 {method}", unit="img")

        for seg_file in pbar:
            orig_filename = seg_file.name.replace(prefix, "")
            orig_path = Path(original_dir) / orig_filename

            if not orig_path.exists():
                stats['failed'] += 1
                continue

            try:
                result = process_image_with_measurement(
                    str(orig_path),
                    str(seg_file),
                    method_output_dir,
                    method,
                    enable_quality_check=enable_quality_check
                )

                if result:
                    # Check if skipped
                    if result.get('skipped', False):
                        stats['skipped'] += 1
                        all_skipped.append(result)

                        # Track skip reasons
                        reason = result['skip_reason']
                        stats['skip_reasons'][reason] = stats['skip_reasons'].get(reason, 0) + 1
                    else:
                        # Valid result
                        results.append(result)
                        stats['success'] += 1

                        if result['coin_detected']:
                            stats['coin_detected'] += 1
                            if result['coin_info'].get('in_corner', False):
                                stats['coin_in_corner'] += 1
                            else:
                                stats['coin_in_center'] += 1
                        else:
                            stats['no_coin'] += 1

                    pbar.set_postfix_str(
                        f"✅{stats['success']} | ⏭️{stats['skipped']} | ❌{stats['failed']}"
                    )
                else:
                    stats['failed'] += 1

            except Exception as e:
                stats['failed'] += 1
                pbar.write(f"❌ Error: {seg_file.name} - {str(e)[:50]}")

        pbar.close()

        # Save results
        if results:
            # JSON
            with open(Path(method_output_dir) / 'measurement_results.json', 'w') as f:
                json.dump(results, f, indent=2)

            # CSV
            flat_results = []
            for r in results:
                flat = {
                    'filename': r['filename'],
                    'method': r['method'],
                    'coin_detected': r['coin_detected'],
                    'coin_in_corner': r['coin_info'].get('in_corner', False) if r['coin_info'] else False,
                    'area_pixels': r['area_info']['area_pixels'],
                    'area_cm2': r['area_info']['area_cm2'],
                    'perimeter_cm': r['area_info']['perimeter_cm'],
                    'width_cm': r['area_info']['width_cm'],
                    'height_cm': r['area_info']['height_cm'],
                    'pixels_per_cm': r['area_info']['pixels_per_cm'],
                    'ratio': r['ratio_info']['ratio'],
                    'percentage': r['ratio_info']['percentage']
                }
                flat_results.append(flat)

            df = pd.DataFrame(flat_results)
            df.to_csv(Path(method_output_dir) / 'measurement_results.csv', index=False)

            all_results[method] = results
            all_stats[method] = stats

        # Print summary
        print(f"\nSummary for {method.upper()}:")
        print(f"   Total: {stats['total']}")
        print(f"   Success: {stats['success']} ({stats['success']/stats['total']*100:.1f}%)")
        print(f"   Skipped: {stats['skipped']} ({stats['skipped']/stats['total']*100:.1f}%)")
        print(f"   Failed: {stats['failed']}")

        if stats['skip_reasons']:
            print(f"\n   Skip Reasons:")
            for reason, count in stats['skip_reasons'].items():
                print(f"      - {reason}: {count}")

    # Save skipped files list
    if all_skipped:
        df_skipped = pd.DataFrame(all_skipped)
        df_skipped.to_csv(Path(output_base_dir) / 'skipped_files.csv', index=False)
        print(f"\nTotal skipped files: {len(all_skipped)}")
        print(f"   Saved to: skipped_files.csv")

    return all_results, all_stats, all_skipped


In [40]:
def create_comparison_report(all_results, all_stats, output_dir):
    """Create comparison report antar methods"""
    print(f"\n{'='*70}")
    print(f"CREATING COMPARISON REPORT")
    print(f"{'='*70}")

    # Combine all results
    combined_data = []

    for method, results in all_results.items():
        for r in results:
            combined_data.append({
                'filename': r['filename'],
                'method': method,
                'coin_detected': r['coin_detected'],
                'area_cm2': r['area_info']['area_cm2'],
                'perimeter_cm': r['area_info']['perimeter_cm'],
                'width_cm': r['area_info']['width_cm'],
                'height_cm': r['area_info']['height_cm'],
                'ratio': r['ratio_info']['ratio']
            })

    df_all = pd.DataFrame(combined_data)

    # Pivot table
    pivot_area = df_all.pivot_table(index='filename', columns='method', values='area_cm2')
    pivot_area.to_csv(Path(output_dir) / 'comparison_area_by_method.csv')

    # Statistics per method
    stats_summary = []
    for method, results in all_results.items():
        areas = [r['area_info']['area_cm2'] for r in results]
        stats_summary.append({
            'method': method,
            'count': len(results),
            'skipped': all_stats[method]['skipped'],
            'success_rate': round(len(results) / all_stats[method]['total'] * 100, 1),
            'avg_area': round(np.mean(areas), 2),
            'std_area': round(np.std(areas), 2),
            'min_area': round(np.min(areas), 2),
            'max_area': round(np.max(areas), 2),
            'coin_detected': all_stats[method]['coin_detected'],
            'coin_in_corner': all_stats[method]['coin_in_corner']
        })

    df_stats = pd.DataFrame(stats_summary)
    df_stats.to_csv(Path(output_dir) / 'comparison_statistics.csv', index=False)

    # Print comparison
    print(f"\nCOMPARISON STATISTICS:")
    print(df_stats.to_string(index=False))

    print(f"\nFiles saved:")
    print(f"   - comparison_area_by_method.csv")
    print(f"   - comparison_statistics.csv")
    print(f"   - skipped_files.csv")

    return df_all, df_stats

In [41]:
start_time = time.time()

METHODS_TO_PROCESS = ['binary_gray', 'binary_hsv', 'opening', 'closing', 'hole_filled']

# Set enable_quality_check=True untuk filtering
all_results, all_stats, all_skipped = batch_area_measurement_multi_method(
    original_dir=AUGMENTED_DIR,
    segmented_dir=SEGMENTED_DIR,
    output_base_dir=MEASUREMENT_DIR,
    methods=METHODS_TO_PROCESS,
    enable_quality_check=True  # ← SET TRUE untuk enable filtering
)

# Create comparison
if all_results:
    df_comparison, df_stats = create_comparison_report(all_results, all_stats, MEASUREMENT_DIR)

elapsed = time.time() - start_time

MULTI-METHOD MEASUREMENT WITH QUALITY FILTERING
Methods to process: 5
Quality check: ENABLED ✅


Processing Method: BINARY_GRAY
Files found: 8464


📏 binary_gray:   0%|          | 0/8464 [00:00<?, ?img/s]


Summary for BINARY_GRAY:
   Total: 8464
   Success: 6956 (82.2%)
   Skipped: 1508 (17.8%)
   Failed: 0

   Skip Reasons:
      - Mask too large (83.6% of image); Image too bright (brightness: 225.6): 3
      - Mask too large (84.0% of image); Image too bright (brightness: 233.1): 1
      - Mask too large (90.7% of image); Image too bright (brightness: 237.0): 3
      - Mask too large (91.2% of image); Image too bright (brightness: 240.6): 1
      - Mask too large (90.7% of image); Image too blurry (variance: 48.1): 1
      - No coin detected: 634
      - Mask too large (83.0% of image); Image too bright (brightness: 235.2): 4
      - Mask too large (83.1% of image); Image too bright (brightness: 235.2): 2
      - Mask too large (85.8% of image); Image too bright (brightness: 240.2); Image too blurry (variance: 47.1): 2
      - Mask too large (82.8% of image); Image too blurry (variance: 36.4): 2
      - Mask too large (90.9% of image); Image too bright (brightness: 240.3); Image too b

📏 binary_hsv:   0%|          | 0/8464 [00:00<?, ?img/s]


Summary for BINARY_HSV:
   Total: 8464
   Success: 7755 (91.6%)
   Skipped: 709 (8.4%)
   Failed: 0

   Skip Reasons:
      - No coin detected: 644
      - No coin detected; Image too dark (brightness: 12.6); Image too blurry (variance: 34.4): 2
      - No coin detected; Image too dark (brightness: 12.6); Image too blurry (variance: 35.4): 1
      - Coin in center (not corner); Image too dark (brightness: 13.8); Image too blurry (variance: 47.4): 1
      - No coin detected; Image too dark (brightness: 9.8); Image too blurry (variance: 25.0): 1
      - No coin detected; Image too blurry (variance: 41.4): 2
      - No coin detected; Image too blurry (variance: 30.1): 1
      - No coin detected; Image too blurry (variance: 35.4): 2
      - No coin detected; Image too blurry (variance: 42.6): 2
      - No coin detected; Image too bright (brightness: 225.7); Image too blurry (variance: 32.1): 2
      - No coin detected; Image too blurry (variance: 49.6): 2
      - No coin detected; Image t

...


Summary for OPENING:
   Total: 8464
   Success: 7755 (91.6%)
   Skipped: 709 (8.4%)
   Failed: 0

   Skip Reasons:
      - No coin detected: 644
      - No coin detected; Image too dark (brightness: 12.6); Image too blurry (variance: 34.4): 2
      - No coin detected; Image too dark (brightness: 12.6); Image too blurry (variance: 35.4): 1
      - Coin in center (not corner); Image too dark (brightness: 13.8); Image too blurry (variance: 47.4): 1
      - No coin detected; Image too dark (brightness: 9.8); Image too blurry (variance: 25.0): 1
      - No coin detected; Image too blurry (variance: 41.4): 2
      - No coin detected; Image too blurry (variance: 30.1): 1
      - No coin detected; Image too blurry (variance: 35.4): 2
      - No coin detected; Image too blurry (variance: 42.6): 2
      - No coin detected; Image too bright (brightness: 225.7); Image too blurry (variance: 32.1): 2
      - No coin detected; Image too blurry (variance: 49.6): 2
      - No coin detected; Image too 

📏 closing:   0%|          | 0/8464 [00:00<?, ?img/s]


Summary for CLOSING:
   Total: 8464
   Success: 7755 (91.6%)
   Skipped: 709 (8.4%)
   Failed: 0

   Skip Reasons:
      - No coin detected: 644
      - No coin detected; Image too dark (brightness: 12.6); Image too blurry (variance: 34.4): 2
      - No coin detected; Image too dark (brightness: 12.6); Image too blurry (variance: 35.4): 1
      - Coin in center (not corner); Image too dark (brightness: 13.8); Image too blurry (variance: 47.4): 1
      - No coin detected; Image too dark (brightness: 9.8); Image too blurry (variance: 25.0): 1
      - No coin detected; Image too blurry (variance: 41.4): 2
      - No coin detected; Image too blurry (variance: 30.1): 1
      - No coin detected; Image too blurry (variance: 35.4): 2
      - No coin detected; Image too blurry (variance: 42.6): 2
      - No coin detected; Image too bright (brightness: 225.7); Image too blurry (variance: 32.1): 2
      - No coin detected; Image too blurry (variance: 49.6): 2
      - No coin detected; Image too 

📏 hole_filled:   0%|          | 0/8464 [00:00<?, ?img/s]


Summary for HOLE_FILLED:
   Total: 8464
   Success: 7753 (91.6%)
   Skipped: 711 (8.4%)
   Failed: 0

   Skip Reasons:
      - No coin detected: 560
      - No coin detected; Mask too large (85.7% of image): 2
      - No coin detected; Mask too large (81.6% of image): 3
      - No coin detected; Mask too large (85.8% of image): 2
      - No coin detected; Mask too large (81.1% of image): 4
      - No coin detected; Mask too large (83.3% of image): 3
      - No coin detected; Mask too large (83.2% of image): 2
      - No coin detected; Image too dark (brightness: 12.6); Image too blurry (variance: 34.4): 2
      - No coin detected; Image too dark (brightness: 12.6); Image too blurry (variance: 35.4): 1
      - Coin in center (not corner); Image too dark (brightness: 13.8); Image too blurry (variance: 47.4): 1
      - No coin detected; Image too dark (brightness: 9.8); Image too blurry (variance: 25.0): 1
      - No coin detected; Image too blurry (variance: 41.4): 2
      - No coin det

In [43]:
print(f"\n{'='*70}")
print(f"FINAL SUMMARY")
print(f"{'='*70}")
print(f"⏱Total time: {elapsed:.2f}s")
print(f"Methods processed: {len(all_results)}")

total_processed = sum(stats['success'] for stats in all_stats.values())
total_skipped = sum(stats['skipped'] for stats in all_stats.values())
total_all = sum(stats['total'] for stats in all_stats.values())

print(f"\nOverall Statistics:")
print(f"   Total files: {total_all}")
print(f"   Processed: {total_processed} ({total_processed/total_all*100:.1f}%)")
print(f"   Skipped: {total_skipped} ({total_skipped/total_all*100:.1f}%)")
print(f"\nTime saved by skipping: ~{total_skipped * 0.5:.1f}s")
print(f"{'='*70}")


FINAL SUMMARY
⏱Total time: 30294.47s
Methods processed: 5

Overall Statistics:
   Total files: 42320
   Processed: 37974 (89.7%)
   Skipped: 4346 (10.3%)

Time saved by skipping: ~2173.0s


In [ ]:
import shutil   # <-- tambahin import ini kalau belum ada

# Folder khusus untuk gambar yang di-skip (original + mask)
SKIPPED_IMAGES_DIR = os.path.join(MEASUREMENT_DIR, 'skipped_images')
os.makedirs(SKIPPED_IMAGES_DIR, exist_ok=True)

In [47]:
def process_image_with_measurement(image_path, segmented_mask_path, output_dir, method_name, enable_quality_check=True):
    original = cv2.imread(image_path)
    mask = cv2.imread(segmented_mask_path, cv2.IMREAD_GRAYSCALE)
    if original is None or mask is None:
        return None

    if original.shape[:2] != mask.shape[:2]:
        mask = cv2.resize(mask, (original.shape[1], original.shape[0]), interpolation=cv2.INTER_NEAREST)

    pixels_per_cm, coin_info = detect_coin_for_calibration(original)

    if enable_quality_check:
        is_valid, skip_reason, quality_score = check_image_quality(original, mask, coin_info)
        if not is_valid:
            # === BAGIAN BARU: SIMPAN ORIGINAL + MASK KE FOLDER SKIPPED_IMAGES ===
            stem = Path(image_path).stem
            skip_folder = os.path.join(SKIPPED_DIR, f"{stem}_{method_name}")
            os.makedirs(skip_folder, exist_ok=True)

            shutil.copy(image_path, os.path.join(skip_folder, Path(image_path).name))
            shutil.copy(segmented_mask_path, os.path.join(skip_folder, Path(segmented_mask_path).name))

            with open(os.path.join(skip_folder, "skip_reason.txt"), "w") as f:
                f.write(f"Method: {method_name}\n")
                f.write(f"Reason: {skip_reason}\n")
                f.write(f"Quality Score: {quality_score:.1f}\n")
            # ===========================================================

            return {
                'filename': Path(image_path).name,
                'method': method_name,
                'skipped': True,
                'skip_reason': skip_reason,
                'quality_score': round(quality_score, 2)
            }

    # Kalau lolos quality check → proses normal
    area_info = calculate_wound_area(mask, pixels_per_cm)
    coin_area_cm2 = np.pi * (coin_info['diameter_cm']/2)**2 if coin_info else None
    ratio_info = calculate_wound_ratio(area_info['area_cm2'], coin_area_cm2)

    measurement_result = {
        'filename': Path(image_path).name,
        'method': method_name,
        'skipped': False,
        'coin_detected': coin_info is not None,
        'coin_info': coin_info,
        'area_info': area_info,
        'ratio_info': ratio_info
    }

    vis = create_visualization(original, mask, coin_info, area_info, ratio_info, method_name)
    out_path = Path(output_dir) / f"measured_{Path(image_path).name}"
    cv2.imwrite(str(out_path), vis)
    measurement_result['output_path'] = str(out_path)

    return measurement_result

In [ ]:
# PILIH METHOD YANG MAU DIPROSES
METHODS_TO_PROCESS = ['binary_gray', 'binary_hsv', 'opening', 'closing', 'hole_filled']

# MULAI PENGUKURAN + AUTO-SKIP + SIMPAN KE skipped_images
start_time = time.time()

all_results, all_stats, all_skipped = batch_area_measurement_multi_method(
    original_dir=AUGMENTED_DIR,
    segmented_dir=SEGMENTED_DIR,
    output_base_dir=MEASUREMENT_DIR,
    methods=METHODS_TO_PROCESS,
    enable_quality_check=True   # True = skip gambar jelek & simpan ke folder skipped_images
)

# Bikin laporan perbandingan
if all_results:
    create_comparison_report(all_results, all_stats, MEASUREMENT_DIR)

print(f"\nSELESAI dalam {time.time() - start_time:.1f} detik")
print(f"Gambar yang di-skip otomatis tersimpan di:\n   {SKIPPED_IMAGES_DIR}")

MULTI-METHOD MEASUREMENT WITH QUALITY FILTERING
Methods to process: 5
Quality check: ENABLED ✅


Processing Method: BINARY_GRAY
Files found: 8464


📏 binary_gray:   0%|          | 0/8464 [00:00<?, ?img/s]

❌ Error: binary_gray_burn_wound_2169_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_2169_rotation_90.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_2169_rotation_180.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_2169_brightness_increase.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_2730_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_2730_rotation_90.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_2730_rotation_180.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_2730_brightness_increase.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_2730_brightness_decrease.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_1159_brightness_increase.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_wound_1703_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: binary_gray_burn_woun

📏 binary_hsv:   0%|          | 0/8464 [00:00<?, ?img/s]

❌ Error: binary_hsv_burn_wound_1159_brightness_increase.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_0899_brightness_decrease.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_1185_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_1185_rotation_90.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_1185_rotation_180.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_1185_brightness_decrease.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_0119_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_0119_rotation_90.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_0119_rotation_180.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_0119_brightness_increase.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_1258_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: binary_hsv_burn_wound_1258_rotat

📏 opening:   0%|          | 0/8464 [00:00<?, ?img/s]

❌ Error: opening_burn_wound_1159_brightness_increase.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_0899_brightness_decrease.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_1185_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_1185_rotation_90.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_1185_rotation_180.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_1185_brightness_decrease.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_0119_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_0119_rotation_90.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_0119_rotation_180.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_0119_brightness_increase.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_1258_horizontal_flip.jpg - name 'shutil' is not defined
❌ Error: opening_burn_wound_1258_rotation_90.jpg - name 'shutil' is not de

In [ ]:
print(f"\n{'='*60}")
print(f"                RINGKASAN AKHIR")
print(f"{'='*60}")

total_processed = sum(stats['total'] for stats in all_stats.values())
total_success   = sum(stats['success'] for stats in all_stats.values())
total_skipped   = sum(stats['skipped'] for stats in all_stats.values())
total_failed    = sum(stats['failed'] for stats in all_stats.values())

print(f"Total gambar diproses          : {total_processed}")
print(f"Berhasil diukur (valid)        : {total_success} gambar")
print(f"DI-SKIP karena kualitas jelek  : {total_skipped} gambar   <<<<<<")
print(f"Gagal (error baca file, dll)   : {total_failed}")
print(f"Success rate                   : {total_success/total_processed*100:.1f}%")

print(f"\nDetail per method:")
for method, stats in all_stats.items():
    print(f"  • {method.upper():12} → "
          f"{stats['success']:3} berhasil | "
          f"{stats['skipped']:3} di-skip | "
          f"{stats['total']} total")

print(f"\nSemua gambar yang di-skip sudah disimpan rapi di folder:")
print(f"   {SKIPPED_IMAGES_DIR}")
print(f"{'='*60}")